## This notebook calculates the following Climate Domain Flood metrics: 
    * Average flood insurance payout per number of claims per census tract

In [64]:
import os
import sys
import pandas as pd
import io
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.write_metadata import (
    append_metadata
)
from scripts.utils.file_helpers import (
    pull_csv_from_directory, upload_csv_aws
) 
pd.set_option('display.max_columns', None)

In [65]:
# read in CA census tiger file
ca_tract_county = "s3://ca-climate-index/0_map_data/ca_tracts_county.csv"
ca_tract_county = gpd.read_file(ca_tract_county)
ca_tract_county = ca_tract_county.drop(columns={'field_1', 'geometry'})
ca_tract_county.columns = ca_tract_county.columns.str.lower()
ca_tract_county = ca_tract_county.applymap(lambda s: s.lower() if type(s) == str else s)

C:\Users\jespi\AppData\Local\Temp\ipykernel_2568\4083292558.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ca_tract_county = ca_tract_county.applymap(lambda s: s.lower() if type(s) == str else s)


In [66]:
bucket_name = 'ca-climate-index'
aws_dir = '2a_subset/climate_risk/flood/loss/fema/flood_claims_ca/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

c:\Users\jespi\eagle\carb-climate-index-4\scripts\utils\file_helpers.py:78: DtypeWarning: Columns (7,35,52,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_data)


Saved DataFrame as 'fema_claims_CA_subset.csv'


In [67]:
flood_claim_data = pd.read_csv('fema_claims_CA_subset.csv')

C:\Users\jespi\AppData\Local\Temp\ipykernel_2568\735346701.py:1: DtypeWarning: Columns (7,35,52,61) have mixed types. Specify dtype option on import or set low_memory=False.
  flood_claim_data = pd.read_csv('fema_claims_CA_subset.csv')


In [68]:
flood_claim_data.columns

Index(['agricultureStructureIndicator', 'asOfDate',
       'basementEnclosureCrawlspaceType', 'policyCount',
       'crsClassificationCode', 'dateOfLoss', 'elevatedBuildingIndicator',
       'elevationCertificateIndicator', 'elevationDifference',
       'baseFloodElevation', 'ratedFloodZone', 'houseWorship',
       'locationOfContents', 'lowestAdjacentGrade', 'lowestFloorElevation',
       'numberOfFloorsInTheInsuredBuilding', 'nonProfitIndicator',
       'obstructionType', 'occupancyType', 'originalConstructionDate',
       'originalNBDate', 'amountPaidOnBuildingClaim',
       'amountPaidOnContentsClaim',
       'amountPaidOnIncreasedCostOfComplianceClaim',
       'postFIRMConstructionIndicator', 'rateMethod',
       'smallBusinessIndicatorBuilding', 'totalBuildingInsuranceCoverage',
       'totalContentsInsuranceCoverage', 'yearOfLoss',
       'primaryResidenceIndicator', 'buildingDamageAmount',
       'buildingDeductibleCode', 'netBuildingPaymentAmount',
       'buildingPropertyValu

In [69]:
flood_claim_data.head()

,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,ratedFloodZone,houseWorship,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,primaryResidenceIndicator,buildingDamageAmount,buildingDeductibleCode,netBuildingPaymentAmount,buildingPropertyValue,causeOfDamage,condominiumCoverageTypeCode,contentsDamageAmount,contentsDeductibleCode,netContentsPaymentAmount,contentsPropertyValue,disasterAssistanceCoverageRequired,eventDesignationNumber,ficoNumber,floodCharacteristicsIndicator,floodWaterDuration,floodproofedIndicator,floodEvent,iccCoverage,netIccPaymentAmount,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,nonPaymentReasonContents,nonPaymentReasonBuilding,numberOfUnits,buildingReplacementCost,contentsReplacementCost,replacementCostBasis,stateOwnedIndicator,waterDepth,floodZoneCurrent,buildingDescriptionCode,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,0,2022-11-01T17:29:25.000Z,1.0,1,7.0,2010-01-20T00:00:00.000Z,0,NaN,NaN,NaN,X,0,2.0,NaN,NaN,1.0,0,NaN,1.0,1955-07-01T00:00:00.000Z,1997-12-18T00:00:00.000Z,708.99,0.00,0.0,0,7,0,250000.0,100000.0,2010,1,1708.0,1,708.99,87936.0,4.0,N,NaN,1,0.00,NaN,NaN,NaN,248.0,NaN,0.0,0,Winter storm,30000.0,0.0,60001,60001.0,ALAMEDA COUNTY*,20.0,NaN,1.0,87936.0,NaN,R,0,3.0,X,1.0,0,CA,Currently Unavailable,94546.0,6001.0,6.001431e+09,6.001431e+10,37.7,-122.1,7c7f6e5d-3745-4c58-9787-554e0050e062
1,0,2023-06-12T23:03:51.000Z,NaN,1,7.0,2023-01-16T00:00:00.000Z,1,NaN,NaN,NaN,AO,0,NaN,NaN,NaN,1.0,0,50.0,11.0,1950-09-12T00:00:00.000Z,2018-09-20T00:00:00.000Z,0.00,0.00,0.0,0,RatingEngine,0,250000.0,0.0,2023,0,0.0,5,0.00,64150.0,4.0,N,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,0,California Atmospheric River,30000.0,0.0,60001,60001.0,ALAMEDA COUNTY*,20.0,20.0,1.0,164381.0,NaN,A,0,NaN,AO,1.0,1,CA,Currently Unavailable,94546.0,6001.0,6.001431e+09,6.001431e+10,37.7,-122.1,2ee7a5e0-4f81-44bc-b71a-4123af697a1e
2,0,2023-05-06T14:36:00.000Z,4.0,1,8.0,2023-01-12T00:00:00.000Z,0,NaN,NaN,NaN,X,0,3.0,NaN,NaN,2.0,0,NaN,1.0,1949-02-01T00:00:00.000Z,2019-03-20T00:00:00.000Z,23864.63,0.00,0.0,0,R,0,250000.0,100000.0,2023,1,22550.0,F,23864.63,110255.0,4.0,N,0.0,F,0.00,0.0,0.0,NaN,NaN,NaN,NaN,0,California Atmospheric River,30000.0,0.0,60002,60002.0,"ALAMEDA, CITY OF",97.0,NaN,0.0,137238.0,0.0,R,0,0.0,AE,1.0,0,CA,Currently Unavailable,94502.0,6001.0,6.001428e+09,6.001428e+10,37.7,-122.2,79535cd3-2b47-42c8-9a93-441951491de7
3,0,2023-08-17T15:53:49.000Z,0.0,1,8.0,2022-12-31T00:00:00.000Z,1,NaN,NaN,NaN,AE,0,3.0,NaN,NaN,1.0,0,50.0,12.0,1950-06-01T00:00:00.000Z,2017-08-29T00:00:00.000Z,20731.30,5440.48,0.0,0,RatingEngine,0,250000.0,100000.0,2022,1,9996.0,2,20731.30,374257.0,2.0,N,6440.0,1,5440.48,101337.0,0.0,NaN,NaN,NaN,NaN,0,California Atmospheric River,30000.0,0.0,60002,60002.0,"ALAMEDA, CITY OF",NaN,NaN,2.0,469254.0,109863.0,A,0,0.0,AE,1.0,0,CA,Currently Unavailable,94501.0,6001.0,6.001428e+09,6.001428e+10,37.8,-122.2,07dd66de-b216-4721-b4ab-742a179b06e4
4,0,2020-01-22T16:55:53.000Z,NaN,1,NaN,2018-04-11T00:00:00.000Z,0,1.0,NaN,NaN,X,0,3.0,NaN,NaN,1.0,0,NaN,1.0,1951-01-01T00:00:00.000Z,2015-11-14T00:00:00.000Z,NaN,NaN,NaN,0,7,0,250000.0,100000.0,2018,1,NaN,F,0.00,NaN,4.0,N,791.0,F,0.00,80000.0,0.0,NaN,NaN,NaN,0.0,0,NaN,30000.0,0.0,60002,60002.0,"ALAMEDA, CITY OF",1.0,20.0,1.0,NaN,0.0,A,0,NaN,X,1.0,0,CA,Currently Unavailable,94502.0,6001.0,6.001428e+09,6.001428e+10,37.7,-122.2,3e620912-b947-43af-ae12-a327f888

In [70]:
# select relevant columns to our metrics
columns_keep = [
    'id',
    'countyCode',
    'censusTract',
    'policyCount',
    'yearOfLoss',
    'netIccPaymentAmount',
    'netContentsPaymentAmount',
    'netBuildingPaymentAmount',
    "amountPaidOnBuildingClaim", 
    "amountPaidOnContentsClaim", 
    "amountPaidOnIncreasedCostOfComplianceClaim"
]
flood_claim_columns = flood_claim_data[columns_keep]

# adjust county and tract columns, changing name, data type, and data formatting
flood_claim_columns['countyCode'] = flood_claim_columns['countyCode'].astype(str).str[1:].str.split('.').str[0]
flood_claim_columns = flood_claim_columns.rename(columns={'countyCode':'countyfp'})

flood_claim_columns['censusTract'] = flood_claim_columns['censusTract'].apply(lambda x: '0' + str(int(float(x))) if pd.notnull(x) else x)
flood_claim_columns = flood_claim_columns.rename(columns={'censusTract':'tract'})

# drop duplicates based on event id columns, there are no duplicates
selected_columns = ['id']
flood_claim_drop_duplicates = flood_claim_columns.duplicated(subset=selected_columns, keep='first')
duplicate_count = flood_claim_drop_duplicates.sum()
duplicate_count

duplicate_count

In [ ]:
# drop rows that do not have location identifier
flood_claim_cleaned = flood_claim_columns.dropna(subset=['countyfp', 'tract'], how='all')

# drop rows that do not have a value when summing payout columns
columns_to_sum = ["amountPaidOnBuildingClaim", 
                  "amountPaidOnContentsClaim", 
                  "amountPaidOnIncreasedCostOfComplianceClaim"]

# Create a new dataframe with rows where the sum of the specified columns is non-zero
flood_claim_cleaned = flood_claim_cleaned[flood_claim_cleaned[columns_to_sum].sum(axis=1) != 0]

# Display the new dataframe
print(len(flood_claim_cleaned))
flood_claim_cleaned.head()


34800


,id,countyfp,tract,policyCount,yearOfLoss,netIccPaymentAmount,netContentsPaymentAmount,netBuildingPaymentAmount,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim
0,7c7f6e5d-3745-4c58-9787-554e0050e062,001,06001430700,1,2010,0.0,0.00,708.99,708.99,0.00,0.0
2,79535cd3-2b47-42c8-9a93-441951491de7,001,06001428301,1,2023,0.0,0.00,23864.63,23864.63,0.00,0.0
3,07dd66de-b216-4721-b4ab-742a179b06e4,001,06001428100,1,2022,0.0,5440.48,20731.30,20731.30,5440.48,0.0
5,0505e69e-02f1-4137-bfd0-288b0df6e891,001,06001428100,1,2022,0.0,0.00,23595.26,23595.26,0.00,0.0
6,766beddb-c8ef-4fc8-b540-8a33baa9977f,001,06001421300,1,2022,0.0,0.00,1043.00,1043.00,0.00,0.0


## Metric 1: Average flood insurance payout per number of claims per census tract

In [ ]:
# create a new column summing relevant columns representing how much insurance paid out on these claims
flood_claim_total_cost = flood_claim_cleaned
flood_claim_total_cost['total_insurance_payment'] = flood_claim_cleaned['netIccPaymentAmount'] + flood_claim_cleaned['netContentsPaymentAmount'] + flood_claim_cleaned['netBuildingPaymentAmount']
flood_claim_total_cost

,id,countyfp,tract,policyCount,yearOfLoss,netIccPaymentAmount,netContentsPaymentAmount,netBuildingPaymentAmount,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,total_insurance_payment
0,7c7f6e5d-3745-4c58-9787-554e0050e062,001,06001430700,1,2010,0.0,0.00,708.99,708.99,0.00,0.0,708.99
2,79535cd3-2b47-42c8-9a93-441951491de7,001,06001428301,1,2023,0.0,0.00,23864.63,23864.63,0.00,0.0,23864.63
3,07dd66de-b216-4721-b4ab-742a179b06e4,001,06001428100,1,2022,0.0,5440.48,20731.30,20731.30,5440.48,0.0,26171.78
5,0505e69e-02f1-4137-bfd0-288b0df6e891,001,06001428100,1,2022,0.0,0.00,23595.26,23595.26,0.00,0.0,23595.26
6,766beddb-c8ef-4fc8-b540-8a33baa9977f,001,06001421300,1,2022,0.0,0.00,1043.00,1043.00,0.00,0.0,1043.00
...,...,...,...,...,...,...,...,...,...,...,...,...
52500,5ff87a66-6551-47e6-b7e1-f78da51a7fe2,an,NaN,1,1991,0.0,0.00,652.44,652.44,0.00,0.0,652.44
52501,2f5286c4-ccaf-4bac-a4e8-ac1f5cb0ddd7,an,NaN,1,1984,0.0,415.00,2534.25,2534.25,415.00,0.0,2949.25
52503,248b7245-e7ab-4f9a-9370-b3f948aba4ec,an,NaN,1,1980,0.0,3754.50,18585.91,18585.91,3754.50,0.0,22340.41
52504,48df698a-0e91-4d69-ac93-d31dc094999e,an,NaN,1,1983,0.0,10000.00,14231.17,14231.17,10000.00,0.0,24231.17


In [ ]:
# group the data my tract and year of loss and sum the total cost
flood_claim_cost_grouped = flood_claim_total_cost.groupby(['tract', 'yearOfLoss']).agg({
    'total_insurance_payment': 'sum',            # Sum the policyCount
    'id': 'count'
}).reset_index()

# Display the resulting DataFrame
flood_claim_cost_grouped.head(5)

,tract,yearOfLoss,total_insurance_payment,id
0,020117060510,1997,87535.57,1
1,022003950200,1979,3366.06,1
2,022051020505,1995,23181.01,1
3,022071001735,1995,7213.73,1
4,023007971000,1986,14965.68,1


In [ ]:
# Group by countyCode and calculate the mean for total_insurance_payment
county_cost_averages = flood_claim_cost_grouped.groupby('tract').agg({
    'total_insurance_payment': 'mean',   # Calculate the average rowCount
    'id' : 'sum'
}).reset_index()

# Rename columns for clarity
county_cost_averages = county_cost_averages.rename(columns={
    'total_insurance_payment': 'average_insurance_payout',
    'id':'total_claims'
})

# Display the resulting DataFrame
county_cost_averages.head()

,tract,average_insurance_payout,total_claims
0,020117060510,87535.57,1
1,022003950200,3366.06,1
2,022051020505,23181.01,1
3,022071001735,7213.73,1
4,023007971000,14965.68,1


In [ ]:
county_cost_averages['avg_insurance_payout_per_claim'] = county_cost_averages['average_insurance_payout'] / county_cost_averages['total_claims']

county_cost_averages

,tract,average_insurance_payout,total_claims,avg_insurance_payout_per_claim
0,020117060510,87535.570000,1,87535.570000
1,022003950200,3366.060000,1,3366.060000
2,022051020505,23181.010000,1,23181.010000
3,022071001735,7213.730000,1,7213.730000
4,023007971000,14965.680000,1,14965.680000
...,...,...,...,...
3248,06115040901,61968.446000,9,6885.382889
3249,06115041000,102431.844286,25,4097.273771
3250,06115041100,29643.575000,2,14821.787500
3251,072127008900,1851.870000,1,1851.870000


In [ ]:
# merge with CA census tracts
merged_flood_claims_cost = pd.merge(ca_tract_county, county_cost_averages, on='tract', how='left')
print(len(merged_flood_claims_cost))
merged_flood_claims_cost.tail()

9129


,tract,countyfp,county,average_insurance_payout,total_claims,avg_insurance_payout_per_claim
9124,06059001303,059,orange,NaN,NaN,NaN
9125,06059001304,059,orange,NaN,NaN,NaN
9126,06059001401,059,orange,NaN,NaN,NaN
9127,06013367200,013,contra costa,2111.67,1.0,2111.67
9128,06037578100,037,los angeles,NaN,NaN,NaN


In [ ]:
flood_claims_metric = merged_flood_claims_cost.drop(columns={'countyfp', 'average_insurance_payout', 'total_claims'})
flood_claims_metric

,tract,county,avg_insurance_payout_per_claim
0,06085504321,santa clara,NaN
1,06085504410,santa clara,NaN
2,06085507003,santa clara,NaN
3,06085507004,santa clara,NaN
4,06085502204,santa clara,NaN
...,...,...,...
9124,06059001303,orange,NaN
9125,06059001304,orange,NaN
9126,06059001401,orange,NaN
9127,06013367200,contra costa,2111.67


In [ ]:
non_nan_count = flood_claims_metric['avg_insurance_payout_per_claim'].count()
non_nan_count

2709

In [ ]:
flood_claims_metric.to_csv('climate_flood_cost_metric.csv', index=False)

Function Call

In [ ]:
@append_metadata
def flood_claims_cost_upload(input_csv, export=False, varname=''):
    '''
    Uploads the flood claim and cost metrics to S3 bucket. The metrics are:
    
    * Average flood insurance payout per number of claims per census tract
    
    Data for these metrics are sourced from FEMA's redacted NFIP claims at:
    https://www.google.com/url?q=https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2&sa=D&source=editors&ust=1723749642983941&usg=AOvVaw0-Ri52Pad7wzLYu2eNKABx

    Methods
    -------
    Relevant columns to our data metrics were isolated, renamed, and adjusted for consistency.
    Data was isolated to include data in non-nan census tracts.
    Duplicates were removed based on event ID.
    Only rows that had non-nan values after summing payout columns were retained.
    Data were grouped by tract and year then summed to identify number of events/cost per tract per year.
    Data were grouped again by tract and averaged to identify average cost/number of policies per census tract.
    The payout column was divided by number of claims.
    Parameters
    ----------
    input_csv: string
        csv flood claim/cost data 
    export: True/False boolean
        False = will not upload resulting df containing CAL CRAI flood claim/cost  metrics to AWS
        True = will upload resulting df containing CAL CRAI flood claim/cost metrics to AWS

    Script
    ------
    climate_flood_claims.ipynb

    Note:
    This function assumes users have configured the AWS CLI such that their access key / secret key pair are stored in ~/.aws/credentials.
    See https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html for guidance.
    '''
    print('Data transformation: relevant columns were isolated and renamed')
    print('Data transformation: duplicate entries by event ID were dropped.')
    print('Data transformation: data was grouped by tract & year then summed, then grouped once again and averaged.')
    print('Data transformation: data was merged with California census tracts.') 
 
    if export == True:
        bucket_name = 'ca-climate-index'
        directory = '3_fair_data/index_data'
        export_filename = [input_csv]
        upload_csv_aws(export_filename, bucket_name, directory)

    if export == False:
        print(f'{input_csv} uploaded to AWS.')
 
    if os.path.exists(input_csv):
        os.remove(input_csv)

In [ ]:
input_csvs = ['climate_flood_cost_metric.csv']

varnames = ['climate_fema_nfip_claim_cost']

# Process the data and export
for input_csv, varname in zip(input_csvs, varnames):
    print(f'Processing {input_csv} with varname {varname}')
    flood_claims_cost_upload(input_csv, export=True, varname='test')
    print(f'Completed uploading {input_csv} with varname {varname}')

Processing climate_flood_cost_metric.csv with varname climate_fema_nfip_claim_cost
Completed uploading climate_flood_cost_metric.csv with varname climate_fema_nfip_claim_cost
